Build a baseline model using the weather station data (this approach used all days in 2018 since the data was available).


Then use the model to make predictions for the ENVI-met(micro-climate) data for that specific day and compare it to model predictions using weather station data also for the same day.

# 1. Import

In [1]:
import pandas as pd
# import seaborn as sns
import numpy as np
import pathlib
import pickle
import pathlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
from datetime import datetime
from sklearn import metrics
import csv

import PyQt5

# 2. Import Files

## 2.1 Save csv files as pickle

Only run this once to save our csv data as pickle files

In [2]:
# --> NO NEED TO RUN SAVED AS PICKLE FILES <--
# WEATHER FILES ##

# # AZ PHX Sky Harbor Data #
AZW_15 = pd.read_csv("./Data/AZW_15.csv")

# # ENVIMET DATA #
BPS = []
Fname = []
for path in pathlib.Path("./Data/BPS").iterdir():
    if path.is_file():
        current_file = pd.read_csv(path)
        BPS.append(current_file)
        Fname.append(path.name.replace('.csv', ''))

# # CAMPUS METABOLISM DATA #
metabol14 = []
for path in pathlib.Path('./Data/metabol14').iterdir():
    if path.is_file():
        current_file = pd.read_csv(path)
        metabol14.append(current_file)

# ## Drop last row of EnviMet Data
for i in range(len(BPS)):
    BPS[i] = BPS[i].drop(16)

# ## Save files as pickle
AZW_15.to_pickle("./Data/AZW_15.pkl")

with open('./Data/BPS.pkl', 'wb') as f:
    pickle.dump(BPS, f)

with open('./Data/Fname.pkl', 'wb') as f:
    pickle.dump(Fname, f)

with open('./Data/metabol14.pkl', 'wb') as f:
    pickle.dump(metabol14, f)

## 2.2 Reload Pickle

In [3]:
# We will convert these to df depending on the building we choose
# 2. Load 14 envimet bldgData (14 filtered buildings)
with open('./Data/BPS.pkl', 'rb') as f:
    envi14 = pickle.load(f)

# 3. Load names of BPS files
with open('./Data/Fname.pkl', 'rb') as f:
    Fname = pickle.load(f)

# 4. Load 14 campus metabolism building energy data
with open('./Data/metabol14.pkl', 'rb') as f:
    metabol14 = pickle.load(f)

## 2.3 Choose building name to import

In [4]:
##Print Building Names ##
for i in range(len(Fname)):
    print(Fname[i])
    
bldname = input('Enter building name: ')

for i in range(len(envi14)):
    if bldname == Fname[i]:
        # save 
        envi_bldg = envi14[i]

if bldname not in Fname:
    print("\x1b[31m\"Please enter a valid name from the list above\"\x1b[0m")


Goldwater
Best Hall
COD North
Bio Design Institute B
University Club
Engineering Research Ctr
ISTB 2
Interdisciplinary AB
Health Services
ISTB 5
ISTB 1
Lifescience A_B_D
Bio Design Institute A
ISTB 4
Enter building name: ISTB 1


## 2.4 Create Data Frame

In [5]:
class building:
    
    def __init__(self, bldgname):
        self.bldgname = bldgname
    
    def campusmetabolism(self):
        for i in range(len(metabol14)):
            if metabol14[i]['bldgname'][0] == bldname:
                cmp = metabol14[i]
            elif (metabol14[i]['bldgname'][0] == 'ISTB-5'):
                cmp = metabol14[i]
        return cmp
    
    def envimet(self):
        env = envi_bldg[['Date', 'Time', 'AirTempInFrontOfAllFacades_MEAN', 'RelativeAirHumidityInFrontOfAllFacades_MEAN',
                     'WindSpeedInFrontOfAllFacades_MEAN']]
        
        env = env.rename(columns = {'AirTempInFrontOfAllFacades_MEAN':'Air Temp',
                                    'RelativeAirHumidityInFrontOfAllFacades_MEAN':'Rel Humid',
                                    'WindSpeedInFrontOfAllFacades_MEAN':'Wind Speed'})
        return env


Bldg = building(bldname)
metabol = Bldg.campusmetabolism() # campus metabolism
envimet = Bldg.envimet()          # envimet
print(bldname)

ISTB 1


In [6]:
#read csv and convert to pkl, then read pkl below
# AZW_15 = pd.read_csv("./Data/AZW_15.csv")

# DF for AZ Weather Data (15-min)
weather_station = pd.read_pickle('./Data/AZW_15.pkl')



# 3. Preprocessing

## 3.1 Formatting Date and Time

### a) Envimet dates

In [7]:
# 1. format time
envimet['Time'] = envimet['Time'].str.replace('.',':')
envimet['Time'] = envimet['Time'].str.replace('01','00')

# convert to 24 hour format
envimet['Time'] = pd.to_datetime(envimet['Time']).dt.strftime('%H:%M')

# 2. format date (still in string)
envimet['Date'] = pd.to_datetime(envimet['Date']).dt.strftime('%m/%d/%Y')

# 3. combine date time column as string and set as index
envimet['Date_Time'] = envimet['Date'] + ' ' + envimet['Time']

# 4. Get string type for month and time
envimet['Month'] = envimet['Date_Time'].str[0:2]
envimet['Time'] = envimet['Time'].str.replace(':','')

# 5. Rearrange columns
print(list(envimet.columns))
envimet = envimet[['Date_Time','Month','Time', 'Air Temp', 'Rel Humid' ]]

envimet = envimet.set_index('Date_Time')
print(envimet)

['Date', 'Time', 'Air Temp', 'Rel Humid', 'Wind Speed', 'Date_Time', 'Month']
                 Month  Time   Air Temp  Rel Humid
Date_Time                                         
06/09/2018 05:00    06  0500  30.119833  12.204521
06/09/2018 06:00    06  0600  28.439681  18.227189
06/09/2018 07:00    06  0700  29.212967  19.053349
06/09/2018 08:00    06  0800  31.196816  18.135058
06/09/2018 09:00    06  0900  33.564851  15.471231
06/09/2018 10:00    06  1000  35.369671  11.311710
06/09/2018 11:00    06  1100  36.443335   9.063576
06/09/2018 12:00    06  1200  37.508947   7.840227
06/09/2018 13:00    06  1300  38.480960   7.009446
06/09/2018 14:00    06  1400  39.180197   6.389399
06/09/2018 15:00    06  1500  39.584199   6.057600
06/09/2018 16:00    06  1600  39.794207   5.906971
06/09/2018 17:00    06  1700  39.732764   5.871568
06/09/2018 18:00    06  1800  39.456794   5.906116
06/09/2018 19:00    06  1900  38.867983   6.210788
06/09/2018 20:00    06  2000  38.055714   6.646973


/home/biswas/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  


In [8]:
# # 5. Get numeric for month hour and minute
# microclimate.Date_Time = pd.to_datetime(microclimate.Date_Time)
# microclimate['Month_num'] = microclimate.Date_Time.dt.month
# microclimate['Hour_num'] = microclimate.Date_Time.dt.hour
# microclimate['Minute_num'] = microclimate.Date_Time.dt.minute
# microclimate.Date_Time = pd.to_datetime(microclimate.Date_Time).dt.strftime('%m/%d/%Y %H:%M')

# # 6. Rearrange columns
# print(list(microclimate.columns))
# microclimate = microclimate[['Date_Time','Month','Time','Month_num', 'Hour_num', 'Minute_num', 'Air Temp', 'Rel Humid' ]]

### b) Weather Station dates

In [9]:
# 1. Get string type for month and time
weather_station['Month'] = weather_station['Date_Time'].str[0:2]
weather_station['Time'] = weather_station['Date_Time'].str[11:]
weather_station['Time'] = weather_station['Time'].str.replace(':','')

# 2. Get numeric for month hour and minute
weather_station.Date_Time = pd.to_datetime(weather_station.Date_Time)
weather_station['Month_num'] = weather_station.Date_Time.dt.month
weather_station['Hour_num'] = weather_station.Date_Time.dt.hour
weather_station['Minute_num'] = weather_station.Date_Time.dt.minute
weather_station.Date_Time = pd.to_datetime(weather_station.Date_Time).dt.strftime('%m/%d/%Y %H:%M')

# 3. set date time as index
weather_station = weather_station.set_index('Date_Time') 

### c) Building energy dates

In [10]:
metabol.tstamp = pd.to_datetime(metabol.tstamp).dt.strftime('%m/%d/%Y %H:%M')

# remove unwanted columns
metabol = metabol[['tstamp','KW', 'CHWTON']]

# set date time as index
metabol = metabol.set_index('tstamp')
print(metabol)

                      KW  CHWTON
tstamp                          
01/01/2018 00:00  185.10   21.02
01/01/2018 00:15  188.49   17.12
01/01/2018 00:30  185.19   16.60
01/01/2018 00:45  184.06   14.99
01/01/2018 01:00  187.10   15.77
...                  ...     ...
12/31/2018 22:45  172.90    8.11
12/31/2018 23:00  172.37    7.37
12/31/2018 23:15  169.94    7.55
12/31/2018 23:30  172.74    8.11
12/31/2018 23:45  173.24    6.93

[35031 rows x 2 columns]


## 3.2 Append Energy Consumption to Weather Station data

In [11]:
weather_station =  pd.concat([metabol, weather_station], axis = 1, join = "inner")

# rearrange column
weather_station = weather_station[['Month','Time','Month_num', 'Hour_num', 'Minute_num', 'Air Temp', 'Rel Humid', 'KW','CHWTON' ]]

# weather_station.to_csv('./Data/weather_st_numstr') # save data with string and numeric date format

## 3.3 Append Energy Consumption to Microclimate Data

In [12]:

time = []
time.append('0500')
time.append('0600')
time.append('0700')
time.append('0800')
time.append('0900')
time.append('1000')
time.append('1100')
time.append('1200')
time.append('1300')
time.append('1400')
time.append('1500')
time.append('1600')
time.append('1700')
time.append('1800')
time.append('1900')
time.append('2000')

month = []
for i in range(16):
    month.append('06')

date = []
date.append('06/09/2018 05:00')
date.append('06/09/2018 06:00')
date.append('06/09/2018 07:00')
date.append('06/09/2018 08:00')
date.append('06/09/2018 09:00')
date.append('06/09/2018 10:00')
date.append('06/09/2018 11:00')
date.append('06/09/2018 12:00')
date.append('06/09/2018 13:00')
date.append('06/09/2018 14:00')
date.append('06/09/2018 15:00')
date.append('06/09/2018 16:00')
date.append('06/09/2018 17:00')
date.append('06/09/2018 18:00')
date.append('06/09/2018 19:00')
date.append('06/09/2018 20:00')
Int_AB_CHWTON = []
x = 15170
for i in range(16):
    Int_AB_CHWTON.append(metabol['CHWTON'][i + x])
    x = x + 3
Int_AB_KW = []
x = 15170
for i in range(16):
    Int_AB_KW.append(metabol['KW'][i + x])
    x = x + 3
Int_AB_June_9 = pd.DataFrame()
Int_AB_June_9['month'] = month
Int_AB_June_9['time'] = time
Int_AB_June_9['CHWTON'] = Int_AB_CHWTON
Int_AB_June_9['KW'] = Int_AB_KW
Int_AB_June_9['date'] = date
Int_AB_June_9['Date_Time'] = Int_AB_June_9['date'] + ' ' + Int_AB_June_9['time']
Int_AB_June_9 = Int_AB_June_9.set_index(['Date_Time'])
# print(Int_AB_June_9)
# Int_AB_June_9['CHWTON']
# envimet = envimet[['Air Temp']]
# Int_AB_June_9[['Air Temp']] = envimet
# microclimate = pd.concat()
Int_AB_June_9 = Int_AB_June_9[['CHWTON']]
Int_AB_June_9 = Int_AB_June_9.set_index(envimet.index)
envimet['CHWTON']= Int_AB_June_9

# 4. Train Models on both datasets 
## 4.1 Define Generalized Model Functions

In [108]:
#using single layer ANN. Model Parameters: hidden layer- 10 neurons, output layer- 1 neuron
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense
from tensorflow.keras import initializers
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.optimizers import SGD
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import backend as K
def prepare_data(dataset):
    Y = dataset['CHWTON']
    X = dataset[['Air Temp', 'Rel Humid']]
    Y= Y.values.reshape(-1, 1)
    # create scaler
    scaler1 = StandardScaler()
    X_scaled = scaler1.fit_transform(X)
    print(X_scaled.shape)
    # #do for output variables
    scaler2 = StandardScaler()
    Y_scaled = scaler2.fit_transform(Y)
    X_scaled = np.asarray(X_scaled).astype('float32')
    Y_scaled = np.asarray(Y_scaled).astype('float32')
    return X_scaled,Y_scaled
def define_model(activation_type, hidden_layers):
    model = Sequential()
    if activation_type == 'leaky_relu':
        model.add(Dense(10, input_dim=2, kernel_initializer='he_normal'))
        model.add(LeakyReLU(alpha=0.1))
        for i in range(0,(hidden_layers-2)):
            model.add(Dense(8))
            model.add(LeakyReLU(alpha=0.1))
        model.add(Dense(12))
        model.add(LeakyReLU(alpha=0.1))
        model.add(Dense(1, activation='linear'))
    else:
        model.add(Dense(10, input_dim=2, kernel_initializer='he_normal', activation=activation_type))
        for i in range(0,(hidden_layers-2)):
            model.add(Dense(8, activation=activation_type))
        model.add(Dense(12, activation=activation_type))
        model.add(Dense(1, activation='linear'))
    model.summary()
    model.compile(loss='mse', optimizer='adam', metrics = ['mse','mae'])
    return model
def plot_curve(history):
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()
def evaluate(model,X,Y):
    #evaluate on test dataset, add evaluation parameters
    Y_preds = model.predict(X)
    RMSE = np.sqrt(metrics.mean_squared_error(Y, Y_preds))
    test_loss = model.evaluate(X, Y)
    return RMSE, test_loss

(34620, 2)
(16, 2)
Model: "sequential_53"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_207 (Dense)            (None, 10)                30        
_________________________________________________________________
dense_208 (Dense)            (None, 8)                 88        
_________________________________________________________________
dense_209 (Dense)            (None, 12)                108       
_________________________________________________________________
dense_210 (Dense)            (None, 1)                 13        
Total params: 239
Trainable params: 239
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
174/174 [==============================] - 0s 1ms/step - loss: 1.2176 - mse: 1.2176 - mae: 0.8156 - val_loss: 0.1769 - val_mse: 0.1769 - val_mae: 0.3213
Epoch 2/100
174/174 [==============================] - 0s 704us/step 

174/174 [==============================] - 0s 737us/step - loss: 0.0866 - mse: 0.0866 - mae: 0.2236 - val_loss: 0.0850 - val_mse: 0.0850 - val_mae: 0.2216
Epoch 45/100
174/174 [==============================] - 0s 745us/step - loss: 0.0866 - mse: 0.0866 - mae: 0.2236 - val_loss: 0.0849 - val_mse: 0.0849 - val_mae: 0.2221
Epoch 46/100
174/174 [==============================] - 0s 729us/step - loss: 0.0864 - mse: 0.0864 - mae: 0.2230 - val_loss: 0.0841 - val_mse: 0.0841 - val_mae: 0.2223
Epoch 47/100
174/174 [==============================] - 0s 715us/step - loss: 0.0865 - mse: 0.0865 - mae: 0.2236 - val_loss: 0.0841 - val_mse: 0.0841 - val_mae: 0.2207
Epoch 48/100
174/174 [==============================] - 0s 722us/step - loss: 0.0865 - mse: 0.0865 - mae: 0.2230 - val_loss: 0.0848 - val_mse: 0.0848 - val_mae: 0.2241
Epoch 49/100
174/174 [==============================] - 0s 757us/step - loss: 0.0864 - mse: 0.0864 - mae: 0.2231 - val_loss: 0.0841 - val_mse: 0.0841 - val_mae: 0.2199
Epoch

Epoch 93/100
174/174 [==============================] - 0s 843us/step - loss: 0.0855 - mse: 0.0855 - mae: 0.2210 - val_loss: 0.0831 - val_mse: 0.0831 - val_mae: 0.2186
Epoch 94/100
174/174 [==============================] - 0s 789us/step - loss: 0.0856 - mse: 0.0856 - mae: 0.2214 - val_loss: 0.0832 - val_mse: 0.0832 - val_mae: 0.2203
Epoch 95/100
174/174 [==============================] - 0s 726us/step - loss: 0.0856 - mse: 0.0856 - mae: 0.2212 - val_loss: 0.0836 - val_mse: 0.0836 - val_mae: 0.2218
Epoch 96/100
174/174 [==============================] - 0s 688us/step - loss: 0.0858 - mse: 0.0858 - mae: 0.2216 - val_loss: 0.0847 - val_mse: 0.0847 - val_mae: 0.2225
Epoch 97/100
174/174 [==============================] - 0s 687us/step - loss: 0.0856 - mse: 0.0856 - mae: 0.2214 - val_loss: 0.0835 - val_mse: 0.0835 - val_mae: 0.2194
Epoch 98/100
174/174 [==============================] - 0s 690us/step - loss: 0.0855 - mse: 0.0855 - mae: 0.2212 - val_loss: 0.0831 - val_mse: 0.0831 - val_mae:

## 4.2 Prepare Data for feeding into training loop

In [ ]:
X,Y = prepare_data(weather_station)
X_train, X_test, Y_train,Y_test = train_test_split(X, Y, test_size=0.2, random_state=20)
X_limited,Y_limited = prepare_data(envimet)

## 4.3 Hyperparameter Tuning experiments

In [111]:
activation_functions = ['relu','elu','selu','leaky_relu','tanh']
no_hidden_layers = [1,3,5,7,9,11]
row_headings = ['1','3','5','7','9','11']
eval_func = []
for func in activation_functions:
    eval_func_layers = []
    for hl in no_hidden_layers:
        model = define_model(func,hl)
        history = model.fit(X_train, Y_train, batch_size=128, epochs=100,  validation_split=0.2)
        RMSE, _ = evaluate(model, X_test, Y_test)
        eval_func_layers.append(RMSE)
    eval_func.append(eval_func_layers)
print(eval_func)   
tab = pd.DataFrame(eval_func, activation_functions, row_headings)
print(tab)
        


Model: "sequential_103"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_556 (Dense)            (None, 10)                30        
_________________________________________________________________
dense_557 (Dense)            (None, 12)                132       
_________________________________________________________________
dense_558 (Dense)            (None, 1)                 13        
Total params: 175
Trainable params: 175
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
174/174 [==============================] - 0s 1ms/step - loss: 0.4472 - mse: 0.4472 - mae: 0.4981 - val_loss: 0.1489 - val_mse: 0.1489 - val_mae: 0.2964
Epoch 2/100
174/174 [==============================] - 0s 635us/step - loss: 0.1160 - mse: 0.1160 - mae: 0.2631 - val_loss: 0.1022 - val_mse: 0.1022 - val_mae: 0.2482
Epoch 3/100
174/174 [==============================]

174/174 [==============================] - 0s 639us/step - loss: 0.0868 - mse: 0.0868 - mae: 0.2240 - val_loss: 0.0852 - val_mse: 0.0852 - val_mae: 0.2203
Epoch 46/100
174/174 [==============================] - 0s 673us/step - loss: 0.0866 - mse: 0.0866 - mae: 0.2233 - val_loss: 0.0841 - val_mse: 0.0841 - val_mae: 0.2226
Epoch 47/100
174/174 [==============================] - 0s 662us/step - loss: 0.0865 - mse: 0.0865 - mae: 0.2236 - val_loss: 0.0839 - val_mse: 0.0839 - val_mae: 0.2199
Epoch 48/100
174/174 [==============================] - 0s 648us/step - loss: 0.0866 - mse: 0.0866 - mae: 0.2234 - val_loss: 0.0840 - val_mse: 0.0840 - val_mae: 0.2213
Epoch 49/100
174/174 [==============================] - 0s 674us/step - loss: 0.0865 - mse: 0.0865 - mae: 0.2234 - val_loss: 0.0841 - val_mse: 0.0841 - val_mae: 0.2205
Epoch 50/100
174/174 [==============================] - 0s 644us/step - loss: 0.0864 - mse: 0.0864 - mae: 0.2235 - val_loss: 0.0841 - val_mse: 0.0841 - val_mae: 0.2203
Epoch

Epoch 94/100
174/174 [==============================] - 0s 728us/step - loss: 0.0861 - mse: 0.0861 - mae: 0.2231 - val_loss: 0.0840 - val_mse: 0.0840 - val_mae: 0.2220
Epoch 95/100
174/174 [==============================] - 0s 664us/step - loss: 0.0860 - mse: 0.0860 - mae: 0.2229 - val_loss: 0.0837 - val_mse: 0.0837 - val_mae: 0.2185
Epoch 96/100
174/174 [==============================] - 0s 683us/step - loss: 0.0861 - mse: 0.0861 - mae: 0.2225 - val_loss: 0.0840 - val_mse: 0.0840 - val_mae: 0.2232
Epoch 97/100
174/174 [==============================] - 0s 672us/step - loss: 0.0860 - mse: 0.0860 - mae: 0.2230 - val_loss: 0.0836 - val_mse: 0.0836 - val_mae: 0.2201
Epoch 98/100
174/174 [==============================] - 0s 664us/step - loss: 0.0861 - mse: 0.0861 - mae: 0.2230 - val_loss: 0.0835 - val_mse: 0.0835 - val_mae: 0.2206
Epoch 99/100
174/174 [==============================] - 0s 674us/step - loss: 0.0860 - mse: 0.0860 - mae: 0.2228 - val_loss: 0.0837 - val_mse: 0.0837 - val_mae:

Epoch 37/100
174/174 [==============================] - 0s 683us/step - loss: 0.0884 - mse: 0.0884 - mae: 0.2271 - val_loss: 0.0863 - val_mse: 0.0863 - val_mae: 0.2279
Epoch 38/100
174/174 [==============================] - 0s 675us/step - loss: 0.0887 - mse: 0.0887 - mae: 0.2277 - val_loss: 0.0857 - val_mse: 0.0857 - val_mae: 0.2235
Epoch 39/100
174/174 [==============================] - 0s 668us/step - loss: 0.0885 - mse: 0.0885 - mae: 0.2269 - val_loss: 0.0859 - val_mse: 0.0859 - val_mae: 0.2246
Epoch 40/100
174/174 [==============================] - 0s 667us/step - loss: 0.0885 - mse: 0.0885 - mae: 0.2276 - val_loss: 0.0859 - val_mse: 0.0859 - val_mae: 0.2261
Epoch 41/100
174/174 [==============================] - 0s 686us/step - loss: 0.0883 - mse: 0.0883 - mae: 0.2273 - val_loss: 0.0858 - val_mse: 0.0858 - val_mae: 0.2248
Epoch 42/100
174/174 [==============================] - 0s 702us/step - loss: 0.0886 - mse: 0.0886 - mae: 0.2274 - val_loss: 0.0902 - val_mse: 0.0902 - val_mae:

Epoch 86/100
174/174 [==============================] - 0s 680us/step - loss: 0.0878 - mse: 0.0878 - mae: 0.2259 - val_loss: 0.0853 - val_mse: 0.0853 - val_mae: 0.2244
Epoch 87/100
174/174 [==============================] - 0s 681us/step - loss: 0.0878 - mse: 0.0878 - mae: 0.2259 - val_loss: 0.0853 - val_mse: 0.0853 - val_mae: 0.2252
Epoch 88/100
174/174 [==============================] - 0s 682us/step - loss: 0.0878 - mse: 0.0878 - mae: 0.2260 - val_loss: 0.0854 - val_mse: 0.0854 - val_mae: 0.2258
Epoch 89/100
174/174 [==============================] - 0s 670us/step - loss: 0.0875 - mse: 0.0875 - mae: 0.2253 - val_loss: 0.0850 - val_mse: 0.0850 - val_mae: 0.2247
Epoch 90/100
174/174 [==============================] - 0s 701us/step - loss: 0.0877 - mse: 0.0877 - mae: 0.2260 - val_loss: 0.0848 - val_mse: 0.0848 - val_mae: 0.2223
Epoch 91/100
174/174 [==============================] - 0s 685us/step - loss: 0.0879 - mse: 0.0879 - mae: 0.2259 - val_loss: 0.0849 - val_mse: 0.0849 - val_mae:

...

Epoch 89/100
174/174 [==============================] - 0s 741us/step - loss: 0.0869 - mse: 0.0869 - mae: 0.2239 - val_loss: 0.0851 - val_mse: 0.0851 - val_mae: 0.2227
Epoch 90/100
174/174 [==============================] - 0s 738us/step - loss: 0.0865 - mse: 0.0865 - mae: 0.2229 - val_loss: 0.0839 - val_mse: 0.0839 - val_mae: 0.2201
Epoch 91/100
174/174 [==============================] - 0s 740us/step - loss: 0.0867 - mse: 0.0867 - mae: 0.2237 - val_loss: 0.0833 - val_mse: 0.0833 - val_mae: 0.2204
Epoch 92/100
174/174 [==============================] - 0s 751us/step - loss: 0.0867 - mse: 0.0867 - mae: 0.2238 - val_loss: 0.0869 - val_mse: 0.0869 - val_mae: 0.2268
Epoch 93/100
174/174 [==============================] - 0s 736us/step - loss: 0.0866 - mse: 0.0866 - mae: 0.2230 - val_loss: 0.0847 - val_mse: 0.0847 - val_mae: 0.2209
Epoch 94/100
174/174 [==============================] - 0s 777us/step - loss: 0.0867 - mse: 0.0867 - mae: 0.2236 - val_loss: 0.0833 - val_mse: 0.0833 - val_mae:

174/174 [==============================] - 0s 818us/step - loss: 0.0877 - mse: 0.0877 - mae: 0.2252 - val_loss: 0.0846 - val_mse: 0.0846 - val_mae: 0.2243
Epoch 28/100
174/174 [==============================] - 0s 811us/step - loss: 0.0871 - mse: 0.0871 - mae: 0.2243 - val_loss: 0.0843 - val_mse: 0.0843 - val_mae: 0.2223
Epoch 29/100
174/174 [==============================] - 0s 806us/step - loss: 0.0877 - mse: 0.0877 - mae: 0.2251 - val_loss: 0.0846 - val_mse: 0.0846 - val_mae: 0.2206
Epoch 30/100
174/174 [==============================] - 0s 806us/step - loss: 0.0878 - mse: 0.0878 - mae: 0.2258 - val_loss: 0.0872 - val_mse: 0.0872 - val_mae: 0.2199
Epoch 31/100
174/174 [==============================] - 0s 818us/step - loss: 0.0876 - mse: 0.0876 - mae: 0.2251 - val_loss: 0.0841 - val_mse: 0.0841 - val_mae: 0.2224
Epoch 32/100
174/174 [==============================] - 0s 806us/step - loss: 0.0877 - mse: 0.0877 - mae: 0.2253 - val_loss: 0.0868 - val_mse: 0.0868 - val_mae: 0.2214
Epoch

Epoch 76/100
174/174 [==============================] - 0s 785us/step - loss: 0.0869 - mse: 0.0869 - mae: 0.2237 - val_loss: 0.0843 - val_mse: 0.0843 - val_mae: 0.2216
Epoch 77/100
174/174 [==============================] - 0s 801us/step - loss: 0.0869 - mse: 0.0869 - mae: 0.2239 - val_loss: 0.0837 - val_mse: 0.0837 - val_mae: 0.2218
Epoch 78/100
174/174 [==============================] - 0s 815us/step - loss: 0.0868 - mse: 0.0868 - mae: 0.2235 - val_loss: 0.0856 - val_mse: 0.0856 - val_mae: 0.2269
Epoch 79/100
174/174 [==============================] - 0s 806us/step - loss: 0.0869 - mse: 0.0869 - mae: 0.2237 - val_loss: 0.0841 - val_mse: 0.0841 - val_mae: 0.2227
Epoch 80/100
174/174 [==============================] - 0s 819us/step - loss: 0.0866 - mse: 0.0866 - mae: 0.2232 - val_loss: 0.0840 - val_mse: 0.0840 - val_mae: 0.2227
Epoch 81/100
174/174 [==============================] - 0s 795us/step - loss: 0.0866 - mse: 0.0866 - mae: 0.2233 - val_loss: 0.0842 - val_mse: 0.0842 - val_mae:

Epoch 13/100
174/174 [==============================] - 0s 853us/step - loss: 0.0896 - mse: 0.0896 - mae: 0.2273 - val_loss: 0.0850 - val_mse: 0.0850 - val_mae: 0.2255
Epoch 14/100
174/174 [==============================] - 0s 874us/step - loss: 0.0891 - mse: 0.0891 - mae: 0.2267 - val_loss: 0.0890 - val_mse: 0.0890 - val_mae: 0.2308
Epoch 15/100
174/174 [==============================] - 0s 844us/step - loss: 0.0887 - mse: 0.0887 - mae: 0.2263 - val_loss: 0.0849 - val_mse: 0.0849 - val_mae: 0.2219
Epoch 16/100
174/174 [==============================] - 0s 816us/step - loss: 0.0887 - mse: 0.0887 - mae: 0.2260 - val_loss: 0.0865 - val_mse: 0.0865 - val_mae: 0.2191
Epoch 17/100
174/174 [==============================] - 0s 811us/step - loss: 0.0894 - mse: 0.0894 - mae: 0.2268 - val_loss: 0.0854 - val_mse: 0.0854 - val_mae: 0.2220
Epoch 18/100
174/174 [==============================] - 0s 821us/step - loss: 0.0888 - mse: 0.0888 - mae: 0.2266 - val_loss: 0.0844 - val_mse: 0.0844 - val_mae:

Epoch 62/100
174/174 [==============================] - 0s 839us/step - loss: 0.0876 - mse: 0.0876 - mae: 0.2245 - val_loss: 0.0838 - val_mse: 0.0838 - val_mae: 0.2183
Epoch 63/100
174/174 [==============================] - 0s 861us/step - loss: 0.0879 - mse: 0.0879 - mae: 0.2250 - val_loss: 0.0849 - val_mse: 0.0849 - val_mae: 0.2224
Epoch 64/100
174/174 [==============================] - 0s 817us/step - loss: 0.0875 - mse: 0.0875 - mae: 0.2239 - val_loss: 0.0856 - val_mse: 0.0856 - val_mae: 0.2169
Epoch 65/100
174/174 [==============================] - 0s 819us/step - loss: 0.0876 - mse: 0.0876 - mae: 0.2243 - val_loss: 0.0879 - val_mse: 0.0879 - val_mae: 0.2221
Epoch 66/100
174/174 [==============================] - 0s 838us/step - loss: 0.0883 - mse: 0.0883 - mae: 0.2254 - val_loss: 0.0843 - val_mse: 0.0843 - val_mae: 0.2199
Epoch 67/100
174/174 [==============================] - 0s 844us/step - loss: 0.0878 - mse: 0.0878 - mae: 0.2246 - val_loss: 0.0842 - val_mse: 0.0842 - val_mae:

## 4.4 Train Model with Optimized hyperparameters

In [ ]:
base_model = define_model('relu',3)
history = base_model.fit(X_train, Y_train, batch_size=128, epochs=100, verbose=1, validation_split=0.2)

Model: "sequential_133"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_771 (Dense)            (None, 10)                30        
_________________________________________________________________
dense_772 (Dense)            (None, 8)                 88        
_________________________________________________________________
dense_773 (Dense)            (None, 12)                108       
_________________________________________________________________
dense_774 (Dense)            (None, 1)                 13        
Total params: 239
Trainable params: 239
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
174/174 [==============================] - 0s 1ms/step - loss: 0.2485 - mse: 0.2485 - mae: 0.3821 - val_loss: 0.1007 - val_mse: 0.1007 - val_mae: 0.2458
Epoch 2/100
174/174 [==============================] - 0s 676us/step - loss: 0.0994 - m

174/174 [==============================] - 0s 680us/step - loss: 0.0855 - mse: 0.0855 - mae: 0.2218 - val_loss: 0.0831 - val_mse: 0.0831 - val_mae: 0.2194
Epoch 45/100
174/174 [==============================] - 0s 720us/step - loss: 0.0856 - mse: 0.0856 - mae: 0.2219 - val_loss: 0.0835 - val_mse: 0.0835 - val_mae: 0.2184
Epoch 46/100
174/174 [==============================] - 0s 656us/step - loss: 0.0856 - mse: 0.0856 - mae: 0.2218 - val_loss: 0.0843 - val_mse: 0.0843 - val_mae: 0.2198
Epoch 47/100
174/174 [==============================] - 0s 652us/step - loss: 0.0855 - mse: 0.0855 - mae: 0.2217 - val_loss: 0.0833 - val_mse: 0.0833 - val_mae: 0.2191
Epoch 48/100
174/174 [==============================] - 0s 684us/step - loss: 0.0857 - mse: 0.0857 - mae: 0.2218 - val_loss: 0.0840 - val_mse: 0.0840 - val_mae: 0.2197
Epoch 49/100
174/174 [==============================] - 0s 724us/step - loss: 0.0855 - mse: 0.0855 - mae: 0.2216 - val_loss: 0.0833 - val_mse: 0.0833 - val_mae: 0.2207
Epoch

Epoch 93/100
174/174 [==============================] - 0s 720us/step - loss: 0.0850 - mse: 0.0850 - mae: 0.2204 - val_loss: 0.0827 - val_mse: 0.0827 - val_mae: 0.2175
Epoch 94/100
174/174 [==============================] - 0s 712us/step - loss: 0.0850 - mse: 0.0850 - mae: 0.2206 - val_loss: 0.0831 - val_mse: 0.0831 - val_mae: 0.2192
Epoch 95/100
174/174 [==============================] - 0s 706us/step - loss: 0.0850 - mse: 0.0850 - mae: 0.2209 - val_loss: 0.0826 - val_mse: 0.0826 - val_mae: 0.2168
Epoch 96/100
174/174 [==============================] - 0s 702us/step - loss: 0.0851 - mse: 0.0851 - mae: 0.2204 - val_loss: 0.0831 - val_mse: 0.0831 - val_mae: 0.2211
Epoch 97/100
174/174 [==============================] - 0s 708us/step - loss: 0.0851 - mse: 0.0851 - mae: 0.2213 - val_loss: 0.0826 - val_mse: 0.0826 - val_mae: 0.2157
Epoch 98/100
 94/174 [===============>..............] - ETA: 0s - loss: 0.0855 - mse: 0.0855 - mae: 0.2210

## 4.5 Evaluate Base Model with Big Dataset and Limited Dataset

In [ ]:
plot_curve(history)
RMSE, _ = evaluate(base_model, X_test, Y_test)
RMSE_base, _ = evaluate(base_model, X_limited,Y_limited)
print(RMSE)
print(RMSE_base)

## 4.6 Transfer features and Fine Tune Base Model on Limited Dataset

In [67]:
def fine_tune(model):
    model.trainable = True
    print("Number of layers in the base model: ", len(base_model.layers))
    # Fine-tune from this layer onwards
    fine_tune_at = 1
    # Freeze all the layers before the `fine_tune_at` layer
    for layer in model.layers[:fine_tune_at]:
      layer.trainable =  False
    model.summary()
    model.compile(loss='mse', optimizer='adam', metrics = ['mse','mae'])
    return model
fine_tuned_model = fine_tune(base_model)
X_limited,Y_limited = prepare_data(envimet)
history = fine_tuned_model.fit(X_limited,Y_limited, epochs=50, verbose=1)

Number of layers in the base model:  4
Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_24 (Dense)             (None, 12)                36        
_________________________________________________________________
dense_25 (Dense)             (None, 8)                 104       
_________________________________________________________________
dense_26 (Dense)             (None, 10)                90        
_________________________________________________________________
dense_27 (Dense)             (None, 1)                 11        
Total params: 241
Trainable params: 205
Non-trainable params: 36
_________________________________________________________________
(16, 2)
Epoch 1/50
1/1 [==============================] - 0s 757us/step - loss: 0.3919 - mse: 0.3919 - mae: 0.5261
Epoch 2/50
1/1 [==============================] - 0s 806us/step - loss: 0.3882 - mse: 0.3882 - mae:

## 4.7 Evaluate Fine-tuned Model on Limited Dataset

In [68]:
RMSE_fine_tune ,_= evaluate(fine_tuned_model,X_limited,Y_limited)
print(RMSE_fine_tune)

1/1 [==============================] - 0s 726us/step - loss: 0.2657 - mse: 0.2657 - mae: 0.4237
0.51549023
